**Recuerde no agregar o quitar celdas en este notebook, ni modificar su tipo. Si lo hace, el sistema automaticamente lo calificará con cero punto cero (0.0)**

Calcule los valores máximo y mínimo de la tercera columna por letra.

In [1]:
%%writefile input.txt
B   1999-08-28   14.1
E   1999-12-06   12.2
E   1993-07-21   17.0
C   1991-02-12   13.8
E   1995-04-25   16.9
A   1992-08-22   14.8
B   1999-06-11   12.1
E   1993-01-27   13.2
E   1999-09-10   11.3
E   1990-05-03   16.4
E   1994-02-14   10.5
A   1988-04-27   12.6
A   1990-10-06   10.7
E   1985-02-12   16.8
E   1998-09-14   16.9
B   1994-08-30   17.0
A   1997-12-15   13.6
B   1995-08-23   10.7
B   1998-11-22   13.8
B   1997-04-09   14.9
E   1993-12-27   18.4
E   1999-01-14   15.3
A   1992-09-19   18.2
B   1993-03-02   14.4
B   1999-10-21   13.5
A   1990-08-31   12.6
C   1994-01-25   10.7
E   1990-02-09   18.8
A   1990-09-26   14.9
A   1993-05-08   16.8
B   1995-09-06   14.7
E   1991-02-18   14.6
A   1993-01-11   14.5
A   1990-07-22   18.4
C   1994-09-09   15.3
C   1994-07-27   10.2
D   1990-10-10   15.1
A   1990-09-05   11.0
B   1991-10-01   15.0
A   1994-10-25   13.1

Writing input.txt


## Mapper

In [2]:
%%writefile mapper.py
#! /usr/bin/env python

import sys

class Mapper:
    
    NOTHING      = 0
    JUST_COUNTER = 1
    JUST_STATUS  = 2
    ALL          = 3
    
    def __init__(s, str_in, str_out, str_err, log_level=NOTHING):
        s.str_in  = str_in
        s.str_out = str_out
        s.str_err = str_err
        s.log_level = log_level
    
    def emit(s, key, value=1):
        s.str_out.write('{}\t{}\n'.format(key, value))

    def status(s, message):
        s.str_err.write('reporter:status: {}\n'.format(message))
    
    def counter(s, counter, amount=1, group='ApplicationCounter'):
        s.str_err.write('reporter:counter: {},{},{}\n'.format(group, counter, amount))
    
    def map(s):
        
        counter = 0
        
        for line in s:
            
            counter += 1
            if s.log_level in [2, 3]: s.status('Processing line {}'.format(line))
            if s.log_level in [1, 3]: s.counter('NumberLines', counter)
            
            s.emit(line[0], line[1])

    def __iter__(s):
        for line in s.str_in:
            yield line[0], line[17:21]
            

if __name__ == "__main__": 
    
    mapper = Mapper(sys.stdin, sys.stdout, sys.stderr)
    
    mapper.map()

Writing mapper.py


## Reducer

In [3]:
%%writefile reducer.py
#!/usr/bin/env python

import sys
from itertools import groupby

class Reducer:
    
    def __init__(s, str_in, str_out, str_err):
        s.str_in  = str_in
        s.str_out = str_out
        s.str_err = str_err
    
    def emit(s, key, mx, mn):
        s.str_out.write('{}\t{}\t{}\n'.format(key, mx, mn))
    
    def reduce(s):
        for key, group in groupby(s, lambda x: x[0]):
            first = True
            for key1, value in group:
                if first:
                    mx = value
                    mn = value
                    first = False
                else:
                    mx = max(mx,value)
                    mn = min(mn, value)
            s.emit(key, mx, mn)
    
    def __iter__(s):
        for line in s.str_in:
            key, val = line.split('\t')
            val = float(val)
            yield(key,val)

if __name__ == '__main__': 
    reducer = Reducer(sys.stdin,sys.stdout,sys.stderr)
    reducer.reduce()

Writing reducer.py


## Ejecución

In [4]:
%%bash
rm -rf output
STREAM=$HADOOP_HOME/share/hadoop/tools/lib/hadoop-streaming-*.jar
chmod +x mapper.py
chmod +x reducer.py
hadoop jar $STREAM -input input.txt -output output  -mapper mapper.py -reducer reducer.py
cat output/part-00000

A	18.4	10.7
B	17.0	10.7
C	15.3	10.2
D	15.1	15.1
E	18.8	10.5


In [5]:
!rm -rf mapper.py reducer.py output input.txt

---

Para realizar la evaluación automática de este libro:

* Abra un Terminal.
* Asegurece que esat en la misma carpeta que contiene este notebook.
* Salve el notebook.
* Ejecute el siguiente comando:

      ./gradetool 07-Taller.ipynb

---